In [1]:
import numpy as np
import sys
sys.executable

'/mnt/c/Users/marc_/OneDrive/ETH/MSC_Thesis/inf-extr/resources/inf-extr-BlECHwmE-py3.10/bin/python'

In [8]:
import torch
torch.cuda.is_available()

True

In [7]:
from flash_attn import flash_attn_qkvpacked_func, flash_attn_func